# Hamming Code Error Correction
This noteboook shows how to use the Hamming code error corrector.

Start by importing necessary modules:

In [1]:
from ecc.hamming import Hamming
from ecc.message import Message
from pydantic import ValidationError

### Message ###

The message module provides the class Message which allows to convert from strings to bits and back using the desired encoding scheme (which defaults to utf-8). To encode a string use the method `to_bits`, and to decode use `from_bits`.


In [2]:
msg = Message('ascii')
bits = msg.to_bits('A message.')
print(bits)
string = msg.from_bits(bits)
print(string)

bitarray('01000001001000000110110101100101011100110111001101100001011001110110010100101110')
A message.


The Message class also provides the method `negate` to negate specified bits of a bit array. This can be used to simulate a message that has been received with noise and hence it is wrongly interpreted by the receiver. For example, suppose we want to alter bits number 3,8,11 and the last three:

In [3]:
print(bits)
print(msg.negate(bits,3,8,11,-3,-2,-1))

bitarray('01000001001000000110110101100101011100110111001101100001011001110110010100101110')
bitarray('01010001101100000110110101100101011100110111001101100001011001110110010100101001')


### Hamming codes ###

This notebook is not intended to teach how Hamming codes work. They are fairly simple, and if a recap is needed, a great explanation is given in [3blue1brown](https://youtu.be/X8jsijhllIA).

To use the hamming code error correction specify the desired number of parity bits. Recall that with $m$ parity bits per block, the maximum data bits are ${k=2^{m}-m-1}$. Therefore, `Hamming(4)` results in what is known as a Hamming(15,11) code. 

Remember that $m\ge{2}$ and as it increases the efficiency rate also increases. Hamming codes can correct only one error per block. More than one can be detected, but not corrected, so keep in mind that increasing $m$ too much is not desired as the block length also increases.

In practice, errors tend to come in burst, so once the blocks are encoded, they are interlaced in a way that bits of an effected block during the transmission channel do not actually have to belong to the same encoded hamming block.   

Play around with the cell below, changing the message, the amount of parity bits choosen, and the bits altered once the message is encoded and ready to be sent or stored. Notice that the length of the encoded message depends on $m$, so draw your own conclusions regarding efficiency rate, bandwidth rate and error rate.

In [4]:
try:
    msg = Message()
    message = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Tempus iaculis urna id volutpat. Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit.'
    print(f'Original message: \n{message}\n')
    bits = msg.to_bits(message)
    ham = Hamming(4)
    enc_bits = ham.encode(bits)
    print(f'Length of encoded message {len(enc_bits)}.\n')
    altered = msg.negate(enc_bits, 10,11,12,13,14,15,16,17,108,109,110,111,112,113,114,115, 1020,1021,1022,1023,1024,-8,-7-6,-5,-4,-3,-2-1)
    decoded = ham.decode(altered)
    decoded_msg = msg.from_bits(decoded)
    print(f'Decoded message: \n{decoded_msg}\n')
    print(f'Hash of original message: {hash(message)}')
    print(f'Hash of decoded message:  {hash(decoded_msg)}')
except ValidationError as exc:
    print(exc)

Original message: 
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Tempus iaculis urna id volutpat. Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit.

Length of encoded message 2624.

Decoded message: 
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Tempus iaculis urna id volutpat. Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit.

Hash of original message: 4185382622779063449
Hash of decoded message:  4185382622779063449


Eventhough a block with more than one error is not corrected by Hamming, sometimes the string decoding yields the expected result, this happens when there are only two wrong bits in a block and one corresponds to the first bit of the block.

In [5]:
message = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Tempus iaculis urna id volutpat. Lacus suspendisse faucibus interdum posuere lorem ipsum dolor sit.'
bits = msg.to_bits(message)
ham = Hamming(7)
enc_bits = ham.encode(bits)
# For the given message and the given hamming code there are 16 blocks of 128 bits each.
# bits 0,10,11 and 12 correspond to the first bits of blocks 0,10,11 and 12 respectively.
# bit 26 corresponds to the second bit of block 10.
# bit 43 corresponds to the third bit of block 11.
# bit 60 corresponds to the third bit of block 12.
# bit 2032 corresponds to the last bit of block 0.
altered = msg.negate(enc_bits,0, 10,11,12,26,43,60,2032)
decoded = ham.decode(altered)
decoded_msg = msg.from_bits(decoded)
print(f'Hash of original message: {hash(message)}')
print(f'Hash of decoded message : {hash(decoded_msg)}')


ERROR!!! 
More than one error detected in same block. Unable to correct.


ERROR!!! 
More than one error detected in same block. Unable to correct.


ERROR!!! 
More than one error detected in same block. Unable to correct.


ERROR!!! 
More than one error detected in same block. Unable to correct.

Hash of original message: 4185382622779063449
Hash of decoded message : 4185382622779063449
